In [14]:
import pandas as pd
import mysql.connector

# MySQL-Verbindung herstellen
connection_params = {
    'host': 'localhost',
    'user': 'admin',
    'password': 'Criminal1234',
    'database': 'CriminalDataDB',
    'allow_local_infile': True
}

conn = mysql.connector.connect(**connection_params)
cursor = conn.cursor()

query = "SELECT * FROM Bevoelkerungsdichte bd JOIN Bevoelkerungsdaten bvd ON bd.GEBIET_NAME = bvd.gemeinde AND bd.INDIKATOR_JAHR = bvd.jahr JOIN Einkommensdaten ed ON bd.BFS_NR = ed.BFS_NR AND bd.GEBIET_NAME = ed.GEBIET_NAME AND bd.INDIKATOR_JAHR = ed.INDIKATOR_JAHR JOIN Finanzvermoegen fv ON bd.BFS_NR = fv.BFS_NR AND bd.GEBIET_NAME = fv.GEBIET_NAME AND bd.INDIKATOR_JAHR = fv.INDIKATOR_JAHR JOIN Einbrueche eb ON bd.GEBIET_NAME = eb.Gemeindename AND bd.INDIKATOR_JAHR = eb.Ausgangsjahr AND bd.BFS_NR = eb.Gemeinde_BFS_Nr"
cursor.execute(query)

# Ergebnisse abrufen und in DataFrame laden
results = cursor.fetchall()
columns = [i[0] for i in cursor.description]
df = pd.DataFrame(results, columns=columns)

# Verbindung schließen
cursor.close()
conn.close()

df.drop(['INDIKATOR_JAHR', 'GEBIET_NAME', 'BFS_NR','gemeinde','jahr'], axis=1, inplace=True)

# Spalten "Ausgangsjahr", "Gemeinde_BFS_Nr" und "Gemeindename" am Anfang des DataFrames einfügen
cols_to_move = ['Ausgangsjahr', 'Gemeinde_BFS_Nr', 'Gemeindename']
df = pd.concat([df[cols_to_move], df.drop(cols_to_move, axis=1)], axis=1)


# DataFrame nach 'Gemeinde_BFS_Nr' aufsteigend sortieren
df.sort_values(by=['Gemeinde_BFS_Nr', 'Ausgangsjahr'], inplace=True)
print(df)


      Ausgangsjahr  Gemeinde_BFS_Nr  Gemeindename  \
0             2010                1        Aeugst   
1             2010                1        Aeugst   
90            2011                1        Aeugst   
91            2011                1        Aeugst   
180           2012                1        Aeugst   
...            ...              ...           ...   
1855          2020              298  Wiesendangen   
2496          2021              298  Wiesendangen   
2497          2021              298  Wiesendangen   
2912          2022              298  Wiesendangen   
2913          2022              298  Wiesendangen   

                      INDIKATOR_NAME INDIKATOR_VALUE  \
0     Bevölkerungsdichte [Einw./km2]           231.5   
1     Bevölkerungsdichte [Einw./km2]           231.5   
90    Bevölkerungsdichte [Einw./km2]           242.8   
91    Bevölkerungsdichte [Einw./km2]           242.8   
180   Bevölkerungsdichte [Einw./km2]           248.9   
...                        

In [15]:
import os

directory = 'Cleaned_CSV_files'
current_directory = os.getcwd()

csv_path = os.path.join(current_directory, directory, 'EDA.csv')

# Create the directory if it doesn't exist
if not os.path.exists(os.path.join(current_directory, directory)):
    os.makedirs(os.path.join(current_directory, directory))

df.to_csv(csv_path, index=False)
print(df)

      Ausgangsjahr  Gemeinde_BFS_Nr  Gemeindename  \
0             2010                1        Aeugst   
1             2010                1        Aeugst   
90            2011                1        Aeugst   
91            2011                1        Aeugst   
180           2012                1        Aeugst   
...            ...              ...           ...   
1855          2020              298  Wiesendangen   
2496          2021              298  Wiesendangen   
2497          2021              298  Wiesendangen   
2912          2022              298  Wiesendangen   
2913          2022              298  Wiesendangen   

                      INDIKATOR_NAME INDIKATOR_VALUE  \
0     Bevölkerungsdichte [Einw./km2]           231.5   
1     Bevölkerungsdichte [Einw./km2]           231.5   
90    Bevölkerungsdichte [Einw./km2]           242.8   
91    Bevölkerungsdichte [Einw./km2]           242.8   
180   Bevölkerungsdichte [Einw./km2]           248.9   
...                        